## Generative Latent Replay with GMMs

Code to test latent replay on benchmark problems and compare with proposed generative latent replay strategies i.e. normalising bottleneck representations and sampling from fitted GMM on latent space.

In [1]:
import random
import numpy as np
from matplotlib import pyplot as plt

# ML imports
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torchvision import transforms as T

from avalanche.training import Naive, Replay, plugins
from avalanche.benchmarks.classic import (
    RotatedMNIST,
    PermutedMNIST,
    PermutedOmniglot,
    RotatedOmniglot,
)

# Local imports
import utils
from models import SimpleCNN, SimpleMLP
from strategies import LatentReplay, GenerativeLatentReplay


Hyperparameter settings

In [2]:
# Config
device = utils.get_device()

# Dataset specific attributes
dataset = RotatedMNIST  # PermutedMNIST
n_classes = 10
n_experiences = 3

# Model specification
model = "cnn"
hidden_size = 64
n_hidden_layers = 4

# Frozen backbone
freeze_depth = 2  # assert freeze_depth <= n_hidden_layers
if model == "mlp":
    latent_layer_number = freeze_depth * 3
elif model == "cnn":
    latent_layer_number = freeze_depth * 2

# Hyperparams
lr = 0.001
l2 = 0.0005
momentum = 0.9

n_epochs = 40
train_mb_size = 128
eval_mb_size = 512

replay_buffer_size = 5000
cl_plugins = None  # [plugins.EarlyStoppingPlugin(patience=1, val_stream_name="train_stream")]  # JA: need to set params

eval_every = -1

# Reproducibility
SEED = 109
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)


Building base model

In [3]:
# Model
n = 6

if model == "mlp":
    networks = [SimpleMLP(n_classes, hidden_size, n_hidden_layers) for i in range(n)]
    transform = T.Compose([T.ToTensor(), T.Lambda(torch.flatten)])

elif model == "cnn":
    networks = [SimpleCNN(n_classes, hidden_size, n_hidden_layers) for i in range(n)]
    transform = T.Compose([T.ToTensor()])
    
print(networks[4])


SimpleCNN(
  (features): Sequential(
    (conv0): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu0): ReLU()
    (conv1): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU()
    (conv2): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu2): ReLU()
    (conv3): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): ReLU()
    (MaxPool): AdaptiveMaxPool2d(output_size=1)
    (drop): Dropout(p=0.5, inplace=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): LazyLinear(in_features=0, out_features=10, bias=True)
  )
)


c:\Users\jacob\miniconda3\envs\env-glr\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Building Continual Learning methods for comparison

In [4]:
# Continual learning strategy
gen_lat_replay_strategy = GenerativeLatentReplay(
    model=networks[4],
    rm_sz=replay_buffer_size,
    train_mb_size=train_mb_size,
    train_epochs=n_epochs,
    eval_mb_size=eval_mb_size,
    latent_layer_num=latent_layer_number,
    device=device,
    eval_every=eval_every,
    plugins=cl_plugins,
)

gen_replay_strategy = GenerativeLatentReplay(
    model=networks[5],
    rm_sz=replay_buffer_size,
    train_mb_size=train_mb_size,
    train_epochs=n_epochs,
    eval_mb_size=eval_mb_size,
    latent_layer_num=0,
    device=device,
    eval_every=eval_every,
    plugins=cl_plugins,
)

In [5]:
naive_strategy = Naive(
    model=networks[0],
    optimizer=SGD(networks[0].parameters(), lr=lr, momentum=momentum, weight_decay=l2),
    train_mb_size=train_mb_size,
    train_epochs=n_epochs,
    eval_mb_size=eval_mb_size,
    device=device,
    eval_every=eval_every,
)

replay_strategy = Replay(
    model=networks[1],
    optimizer=SGD(networks[1].parameters(), lr=lr, momentum=momentum, weight_decay=l2),
    train_mb_size=train_mb_size,
    train_epochs=n_epochs,
    eval_mb_size=eval_mb_size,
    device=device,
    criterion=CrossEntropyLoss(),
    eval_every=eval_every,
    plugins=cl_plugins,
)

In [6]:
# Sanity check - shuld perform similar to "Replay"
replay_thawed_strategy = LatentReplay(
    model=networks[2],
    rm_sz=replay_buffer_size,
    train_mb_size=train_mb_size,
    train_epochs=n_epochs,
    eval_mb_size=eval_mb_size,
    latent_layer_num=0,
    device=device,
    subsample_replays=True,
    eval_every=eval_every,
    plugins=cl_plugins,
)

lat_replay_strategy = LatentReplay(
    model=networks[3],
    rm_sz=replay_buffer_size,
    train_mb_size=train_mb_size,
    train_epochs=n_epochs,
    eval_mb_size=eval_mb_size,
    latent_layer_num=latent_layer_number,
    device=device,
    subsample_replays=True,
    eval_every=eval_every,
    plugins=cl_plugins,
)

In [7]:
strategies = {
    #"Naive": {"model": naive_strategy, "results": []},
    #"Replay": {"model": replay_strategy, "results": []},
    #"Replay (re-imp)": {"model": replay_thawed_strategy, "results": []},
    #"Latent Replay": {"model": lat_replay_strategy, "results": []},
    #"Generative Replay": {"model": gen_replay_strategy, "results": []},
    "Generative Latent Replay": {"model": gen_lat_replay_strategy, "results": []},
}

Training loop

In [8]:
for strategy in strategies.keys():
    experiences = dataset(
        n_experiences=n_experiences,
        train_transform=transform,
        eval_transform=transform,
        seed=SEED,
        rotations_list=[0, 60, 300],
    )
    train_stream = experiences.train_stream
    test_stream = experiences.test_stream

    for train_exp in train_stream:
        strategies[strategy]["model"].train(train_exp)
        strategies[strategy]["results"].append(
            strategies[strategy]["model"].eval(train_stream)
        )


-- >> Start of training phase << --
100%|██████████| 469/469 [08:02<00:00,  1.03s/it]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.2991
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.1134
100%|██████████| 469/469 [00:25<00:00, 18.12it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.2810
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.1756
100%|██████████| 469/469 [00:26<00:00, 17.77it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.1935
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2394
100%|██████████| 469/469 [00:27<00:00, 17.07it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.6392
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4461
100%|██████████| 469/469 [00:28<00:00, 16.64it/s]
Epoch 4 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.8892
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7089
100%|██████████| 469/469 [00:29<00:00, 16.06it/s]
Epoch 5 ended.
	Loss_Epoch/tr

ValueError: Found array with dim 4. GaussianMixture expected <= 2.

Plotting

In [ ]:
import importlib
importlib.reload(utils)

fig, axes = plt.subplots(1, (len(strategies)), sharey="row")

utils.plot_multiple_results(
    [s["results"] for s in strategies.values()],
    strategies.keys(),
    axes,
    fig,
    n_experiences,
)
